In [1]:
import pandas as pd
import numpy as np

In [17]:
import lamini
import jsonlines
import torch
import jsonlines
import itertools
from pprint import pprint
import datasets
from datasets import load_dataset
from transformers import BitsAndBytesConfig,AutoTokenizer,AutoModelForCausalLM
from transformers import TrainingArguments

In [18]:
instruction_dataset_df = pd.read_json("lamini_docs.jsonl", lines=True)
instruction_dataset_df

,question,answer
0,What are the different types of documents avai...,"Lamini has documentation on Getting Started, A..."
1,What is the recommended way to set up and conf...,Lamini can be downloaded as a python package a...
2,How can I find the specific documentation I ne...,"You can ask this model about documentation, wh..."
3,Does the documentation include explanations of...,Our documentation provides both real-world and...
4,Does the documentation provide information abo...,External dependencies and libraries are all av...
...,...,...
1395,What is Lamini and what is its collaboration w...,Lamini is a library that simplifies the proces...
1396,How does Lamini simplify the process of access...,Lamini simplifies data access in Databricks by...
1397,What are some of the key features provided by ...,Lamini automatically manages the infrastructur...
1398,How does Lamini ensure data privacy during the...,"During the training process, Lamini ensures da..."


In [19]:
len(instruction_dataset_df)

1400

In [20]:
examples=instruction_dataset_df.to_dict()
text=examples["question"][1] + examples['answer'][1]
pprint(text)

('What is the recommended way to set up and configure the code '
 'repository?Lamini can be downloaded as a python package and used in any '
 'codebase that uses python. Additionally, we provide a language agnostic REST '
 'API. We’ve seen users develop and train models in a notebook environment, '
 'and then switch over to a REST API to integrate with their production '
 'environment.')


In [22]:
len(examples)

2

In [23]:
#creating Q-Answer templates for prompt
prompt_template_qa="""```question:
 {question}

```answer:{answer}"""

In [24]:
#passing the question-answer to prompt template to form raw prompt
question=examples['question'][0]
answer=examples['answer'][0]
text_with_prompt_template=prompt_template_qa.format(question=question,answer=answer)
pprint(text_with_prompt_template)

('```question:\n'
 ' What are the different types of documents available in the repository '
 "(e.g., installation guide, API documentation, developer's guide)?\n"
 '\n'
 '```answer:Lamini has documentation on Getting Started, Authentication, '
 'Question Answer Model, Python Library, Batching, Error Handling, Advanced '
 'topics, and class documentation on LLM Engine available at '
 'https://lamini-ai.github.io/.')


In [10]:
#creating anotherprompt template for only question passing
prompt_template_q="""```question:
 {question}

```answer:"""

In [25]:
#converting all question-answer pairs to format using prompt template
examples_count=len(examples['question'])
finetuning_dataset_text_only=[]
finetuning_dataset_question_answer=[]
for i in range(examples_count):
    question=examples['question'][i]
    answer=examples['answer'][i]
    text_with_prompt_template_qa=prompt_template_qa.format(question=question,answer=answer)
    finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})
    
    text_with_prompt_template_q=prompt_template_q.format(question=question)
    finetuning_dataset_question_answer.append({"question":text_with_prompt_template_q,"answer":answer})

In [26]:
len(finetuning_dataset_text_only)

1400

In [27]:
pprint(finetuning_dataset_text_only[1])

{'text': '```question:\n'
         ' What is the recommended way to set up and configure the code '
         'repository?\n'
         '\n'
         '```answer:Lamini can be downloaded as a python package and used in '
         'any codebase that uses python. Additionally, we provide a language '
         'agnostic REST API. We’ve seen users develop and train models in a '
         'notebook environment, and then switch over to a REST API to '
         'integrate with their production environment.'}


In [28]:
pprint(finetuning_dataset_question_answer[1])

{'answer': 'Lamini can be downloaded as a python package and used in any '
           'codebase that uses python. Additionally, we provide a language '
           'agnostic REST API. We’ve seen users develop and train models in a '
           'notebook environment, and then switch over to a REST API to '
           'integrate with their production environment.',
 'question': '```question:\n'
             ' What is the recommended way to set up and configure the code '
             'repository?\n'
             '\n'
             '```answer:'}


In [29]:
#lets store the above formated data
with jsonlines.open(f'lamini_docs_processed.jsonl','w') as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [30]:
instruction_tuned_dataset=load_dataset("tatsu-lab/alpaca",split="train",streaming=True)
m=5
top_m=list(itertools.islice(instruction_tuned_dataset,m))
for i in top_m:
    pprint(i)

{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that '
         'appropriately completes the request.\n'
         '\n'
         '### Instruction:\n'
         'Give three tips for staying healthy.\n'
         '\n'
         '### Response:\n'
         '1.Eat a balanced diet and make sure to include plenty of fruits and '
         'vegetables. \n'
         '2. Exercise regularly to keep your body active and strong. \n'
         '3. Get enough sleep and maintain a consistent sleep schedule.'}
{'input': '',
 'instruction': 'What are the three primary colors?',
 'output': 'The three primary colors are red, blue, and yellow.',
 'text': 'Below 

In [31]:
prompt_template_withInput="""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
###Instruction:
{instruction}

###Input:
{input}

###Response:"""


prompt_template_without_input=""" Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction
{instruction}

###Response:"""

In [32]:
processed_data=[]
for i in top_m:
    if not i['input']:
        processed_prompt=prompt_template_without_input.format(instruction=i["instruction"])
        #print(processed_prompt)
    else:
        processed_prompt=prompt_template_withInput.format(instruction=i["instruction"],input=i['input'])
        
    processed_data.append({"input":processed_prompt,"output":i['output']})

In [33]:
with jsonlines.open(f'alpaca_processed_jsonl','w')as writer:
    writer.write_all(processed_data)

In [34]:
dataset_path_hf="lamini/alpaca"
dataset_alpaca_hf=load_dataset(dataset_path_hf)
print(dataset_alpaca_hf)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


In [35]:
dataset_alpaca_hf['train'][0]['input']

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:'

In [36]:
text=f"""Tell me how to train my dog"""
prompt=f""" you will be provided with aquestion delimited by three back ticks.
answer them in below format\
step 1-...
step 2-...
...
step N-..
```{text}```

"""

In [38]:
from llama import BasicModelRunner
from lamini import LlamaV2Runner
non_instruct_model = BasicModelRunner("meta-llama/Llama-2-7b-hf",config={"production.key":"ebf6623eb26e2c09134532682f10057307e3f60e"})
non_instruct_output = non_instruct_model("Tell me how to train my dog to sit")
print("Not instruction-tuned output (Llama 2 Base):", non_instruct_output)

Not instruction-tuned output (Llama 2 Base): .
Tell me how to train my dog to sit. I have a 10 month old puppy and I want to train him to sit. I have tried the treat method and the verbal command method. I have tried both and he just doesn't seem to get it. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and pushing him down. I have tried to get him to sit by putting my hand on his back and push

In [39]:
#fine tuned model output
instruct_model = BasicModelRunner("meta-llama/Llama-2-7b-chat-hf", config={"production.key":"ebf6623eb26e2c09134532682f10057307e3f60e"})
instruct_output = instruct_model("Tell me how to train my dog to sit")
print("Instruction-tuned output (Llama 2): ", instruct_output)

Instruction-tuned output (Llama 2):   on command.
Training a dog to sit on command is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement. Here's a step-by-step guide on how to train your dog to sit on command:

1. Choose a quiet and distraction-free area: Find a quiet area with minimal distractions where your dog can focus on you.
2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4. Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5. Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat. The moment their bottom touches the ground, say "good sit" and give them the treat.
6. Repeat the process: Repeat steps 3-5 several times, so your dog starts to associate the command "sit" with


### **FineTuning of a Small Model**

In [40]:
tokenizer=AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model=AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [41]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [42]:
#checking lamini_docs_finetuned model
instruction_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")

In [43]:
#passing the question-answer dataset as input to above lamini finetuned model

pprint(inference('Can Lamini generate technical documentation or user manuals for software projects?', instruction_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


('Yes, Lamini can generate technical documentation or user manuals for '
 'software projects. This can be achieved by providing a prompt for a specific '
 'technical question or question to the LLM Engine, or by providing a prompt '
 'for a specific technical question or question. Additionally, Lamini can be '
 'trained on specific technical questions or questions to help users '
 'understand the process and provide feedback to the LLM Engine. Additionally, '
 'Lamini')


In [44]:
text="Hi, I am Hamza Ali"
#encode text into numbers using tokeinzer
encoded_text=tokenizer(text)["input_ids"]
encoded_text

[12764, 13, 309, 717, 5516, 4019, 14355]

In [45]:
encoded_labels = [12764, 13, 309, 717, 5516, 4019, 14355]
decoded_text = tokenizer.decode(encoded_labels)
print(decoded_text)
print(encoded_labels)


Hi, I am Hamza Ali
[12764, 13, 309, 717, 5516, 4019, 14355]


In [46]:
list_text = ["I am Hamza", "I am a Data Scientist", "I am a student at UET Peshawar"]
encoded_texts = tokenizer(list_text)['input_ids']
encoded_texts

[[42, 717, 5516, 4019],
 [42, 717, 247, 5128, 11615, 382],
 [42, 717, 247, 5974, 387, 530, 2025, 367, 15897, 1403, 274]]

In [47]:
### Just Padding
tokenizer.pad_token = tokenizer.eos_token
encoded_texts = tokenizer(list_text, padding=True)['input_ids']
encoded_texts

[[42, 717, 5516, 4019, 0, 0, 0, 0, 0, 0, 0],
 [42, 717, 247, 5128, 11615, 382, 0, 0, 0, 0, 0],
 [42, 717, 247, 5974, 387, 530, 2025, 367, 15897, 1403, 274]]

In [48]:
### Both padding and Truncation is Here
tokenizer.pad_token = tokenizer.eos_token
encoded_texts = tokenizer(list_text, padding=True, truncation=True, max_length=5)['input_ids']
encoded_texts

[[42, 717, 5516, 4019, 0],
 [42, 717, 247, 5128, 11615],
 [42, 717, 247, 5974, 387]]

In [49]:
filename = "lamini_docs.jsonl"
instruction_dataset_df=pd.read_json(filename,lines=True)
examples=instruction_dataset_df.to_dict()

if "question" in examples and "answer" in examples:
    text=examples['question'][0] + examples['answer'][0]
elif "instruction" in examples and "response" in examples:
    text=examples['instruction'][0] + examples['response'][0]
else:
    text=examples['text'][0]


prompt_template="""### Question:
{question}

### Answer:{answer}"""

num_examples=len(examples["question"])
finetuning_dataset=[]
for i in range(num_examples):
    question=examples['question'][i]
    answer=examples['answer'][i]
    text_with_prompt_template=prompt_template.format(question=question,answer=answer)
    finetuning_dataset.append({"question": text_with_prompt_template,"answer":answer})

from pprint import pprint
print("one datapoint in finetuning dataset")
pprint(finetuning_dataset[0])

one datapoint in finetuning dataset
{'answer': 'Lamini has documentation on Getting Started, Authentication, '
           'Question Answer Model, Python Library, Batching, Error Handling, '
           'Advanced topics, and class documentation on LLM Engine available '
           'at https://lamini-ai.github.io/.',
 'question': '### Question:\n'
             'What are the different types of documents available in the '
             'repository (e.g., installation guide, API documentation, '
             "developer's guide)?\n"
             '\n'
             '### Answer:Lamini has documentation on Getting Started, '
             'Authentication, Question Answer Model, Python Library, Batching, '
             'Error Handling, Advanced topics, and class documentation on LLM '
             'Engine available at https://lamini-ai.github.io/.'}


In [50]:
pprint(finetuning_dataset[0]["question"])

('### Question:\n'
 'What are the different types of documents available in the repository (e.g., '
 "installation guide, API documentation, developer's guide)?\n"
 '\n'
 '### Answer:Lamini has documentation on Getting Started, Authentication, '
 'Question Answer Model, Python Library, Batching, Error Handling, Advanced '
 'topics, and class documentation on LLM Engine available at '
 'https://lamini-ai.github.io/.')


In [51]:
text= finetuning_dataset[0]["question"]+ finetuning_dataset[0]["answer"]
tokenized_inputs=tokenizer(text,
                           padding=True,
                           return_tensors='np')
print(tokenized_inputs['input_ids'])

[[ 4118 19782    27   187  1276   403   253  1027  3510   273  7177  2130
    275   253 18491   313    70    15    72   904 12692  7102    13  8990
  10097    13 13722   434  7102  6177   187   187  4118 37741    27    45
   4988    74   556 10097   327 27669 11075   264    13  5271 23058    13
  19782 37741 10031    13 13814 11397    13   378 16464    13 11759 10535
   1981    13 21798 12989    13   285   966 10097   327 21708    46 10797
   2130   387  5987  1358    77  4988    74    14  2284    15  7280    15
    900 14206    45  4988    74   556 10097   327 27669 11075   264    13
   5271 23058    13 19782 37741 10031    13 13814 11397    13   378 16464
     13 11759 10535  1981    13 21798 12989    13   285   966 10097   327
  21708    46 10797  2130   387  5987  1358    77  4988    74    14  2284
     15  7280    15   900 14206]]


In [52]:
max_length=2048
max_length=min(tokenized_inputs['input_ids'].shape[1],
              max_length,)

In [53]:
examples["question"][0]

"What are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?"

In [55]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    #print("text-"+ text)
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [56]:
finetuning_dataset_loaded=datasets.load_dataset('json',
                                                data_files=filename,
                                                split='train')

In [57]:
finetuning_dataset_loaded

Dataset({
    features: ['question', 'answer'],
    num_rows: 1400
})

In [58]:
tokeinzed_dataset=finetuning_dataset_loaded.map(tokenize_function,
                                               batched=True,
                                               batch_size=1,
                                               drop_last_batch=True)
tokeinzed_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1400
})

In [59]:
tokeinzed_dataset=tokeinzed_dataset.add_column("labels",tokeinzed_dataset['input_ids'])

In [60]:
len(tokeinzed_dataset['labels'][0])

77

In [61]:
len(tokeinzed_dataset[1]['labels'])

75

### **Prepareing for Training**

In [62]:
split_dataset=tokeinzed_dataset.train_test_split(test_size=0.2,shuffle=True,seed=123)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1120
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 280
    })
})

In [63]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

In [64]:
len(split_dataset['train'][0]['labels'])

74

In [65]:
len(split_dataset['train'][0]['input_ids'])

74

In [66]:
import huggingface_hub

In [67]:
dataset_path="lamini_docs.jsonl" 
use_hf=False

In [68]:
dataset_path = "lamini/lamini_docs"
use_hf = True

In [69]:
model_name = "EleutherAI/pythia-70m"

In [70]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [71]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-01-19 12:06:47,169 - DEBUG - utilities - Config: datasets.path: lamini/lamini_docs
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize True lamini/lamini_docs


2024-01-19 12:07:19,002 - DEBUG - fsspec.local - open file: C:/Users/DELL/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json
2024-01-19 12:07:19,040 - DEBUG - fsspec.local - open file: C:/Users/DELL/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [72]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [73]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

2024-01-19 12:07:49,289 - DEBUG - __main__ - Select CPU device


In [74]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): G

In [75]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [76]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
Model's answer: 


I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


In [77]:
max_steps = 3

In [78]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [79]:
from transformers import Trainer
from torch import nn

In [80]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [81]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.36.2', '0.26.0')

In [82]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): G

In [85]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [86]:
training_output = trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

{'loss': 3.3405, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 3.2429, 'learning_rate': 5e-06, 'epoch': 0.01}
{'loss': 3.4016, 'learning_rate': 0.0, 'epoch': 0.01}
{'train_runtime': 11.8086, 'train_samples_per_second': 1.016, 'train_steps_per_second': 0.254, 'train_loss': 3.3283477624257407, 'epoch': 0.01}


### save the model

In [87]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: lamini_docs_3_steps/final


In [88]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [89]:
finetuned_slightly_model.to(device) 

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): G

In [90]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Finetuned slightly model's answer: 


I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamin


##### SO as we see in the above output that the finetned model is not working well beacuse of its training on just 3 data points out of the whole dataset

In [93]:
# now checking what the exactly output looks like:
test_answer = test_dataset[0]['answer']
print("Target answer output (test): \n" , test_answer)

Target answer output (test): 
 Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


### Now run that similar model for two epochs

In [94]:
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
tokenizer = AutoTokenizer.from_pretrained("lamini/lamini_docs_finetuned")

finetuned_longer_model.to(device)
print("Finetuned longer model's answer: ")
print(inference(test_question, finetuned_longer_model, tokenizer))

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned longer model's answer: 
Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini
